# Dynamic
## Dynamic Prompt

In [1]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [2]:
from langchain.chat_models import init_chat_model

model = init_chat_model("openai:gpt-5")

ModuleNotFoundError: No module named 'langchain.globals'

In [3]:
from langchain_core.tools import tool
    
@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"


In [4]:
from typing import TypedDict

class RuntimeContext(TypedDict):
    is_employee: bool


In [5]:
SYSTEM_TEMPLATE = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
{table_limits}
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""


In [ ]:
from langchain.agents.middleware.types import dynamic_prompt, AgentState, ModelRequest
from langgraph.runtime import Runtime

@dynamic_prompt
def dynamic_system_prompt(request: ModelRequest, state: AgentState, runtime: Runtime[RuntimeContext]) -> str:
    
    is_employee = runtime.context.get("is_employee", False)

    if not is_employee:
        table_limits = "- Limit access to these tables: Album, Artist, Genre, Playlist, PlaylistTrack, Track."
    else:
        table_limits = ""
    
    return SYSTEM_TEMPLATE.format(table_limits=table_limits)



In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent = create_agent(
    model="openai:gpt-5",
    tools=[execute_sql],
    middleware=[dynamic_system_prompt],
    context_schema=RuntimeContext
)

In [ ]:
question = "What is the most costly purchase by Frank Harris?"
steps=[]

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context={"is_employee": False},   
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

In [13]:
question = "What is the most costly purchase by Frank Harris?"
steps=[]

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context={"is_employee": True},   
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_xlXpfjawIO2MIXTcCWafht5l)
 Call ID: call_xlXpfjawIO2MIXTcCWafht5l
  Args:
    query: SELECT name, sql FROM sqlite_master WHERE type='table' ORDER BY name LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('Album', 'CREATE TABLE [Album]\n(\n    [AlbumId] INTEGER  NOT NULL,\n    [Title] NVARCHAR(160)  NOT NULL,\n    [ArtistId] INTEGER  NOT NULL,\n    CONSTRAINT [PK_Album] PRIMARY KEY  ([AlbumId]),\n    FOREIGN KEY ([ArtistId]) REFERENCES [Artist] ([ArtistId]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\n)'), ('Artist', 'CREATE TABLE [Artist]\n(\n    [ArtistId] INTEGER  NOT NULL,\n    [Name] NVARCHAR(120),\n    CONSTRAINT [PK_Artist] PRIMARY KEY  ([ArtistId])\n)'), ('Customer', 'CREATE T